In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from google.cloud import bigquery


In [3]:
spark = SparkSession.builder.appName('geoEDA').getOrCreate()
spark.sparkContext.getConf().getAll()

[('spark.stage.maxConsecutiveAttempts', '10'),
 ('spark.dynamicAllocation.minExecutors', '1'),
 ('spark.eventLog.enabled', 'true'),
 ('spark.submit.pyFiles',
  '/root/.ivy2/jars/com.johnsnowlabs.nlp_spark-nlp_2.12-4.4.0.jar,/root/.ivy2/jars/graphframes_graphframes-0.8.2-spark3.1-s_2.12.jar,/root/.ivy2/jars/com.typesafe_config-1.4.2.jar,/root/.ivy2/jars/org.rocksdb_rocksdbjni-6.29.5.jar,/root/.ivy2/jars/com.amazonaws_aws-java-sdk-bundle-1.11.828.jar,/root/.ivy2/jars/com.github.universal-automata_liblevenshtein-3.0.0.jar,/root/.ivy2/jars/com.google.cloud_google-cloud-storage-2.16.0.jar,/root/.ivy2/jars/com.navigamez_greex-1.0.jar,/root/.ivy2/jars/com.johnsnowlabs.nlp_tensorflow-cpu_2.12-0.4.4.jar,/root/.ivy2/jars/it.unimi.dsi_fastutil-7.0.12.jar,/root/.ivy2/jars/org.projectlombok_lombok-1.16.8.jar,/root/.ivy2/jars/com.google.guava_guava-31.1-jre.jar,/root/.ivy2/jars/com.google.guava_failureaccess-1.0.1.jar,/root/.ivy2/jars/com.google.guava_listenablefuture-9999.0-empty-to-avoid-conflict-

In [4]:
bq_client = bigquery.Client(project='msca-bdp-student-ap')


In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, desc, expr

community_areas = spark.read.format("bigquery").option("table", "chicago_rideshare.community_areas").load()
ride_locations = spark.read.format("bigquery").option("table", "chicago_rideshare.geo_rides_2018").load()

result_df = community_areas.join(
    ride_locations,
    expr("ST_Contains(ST_GeomFromWKT(community_areas.geometry), ST_GeomFromWKT(ride_locations.pickup_geo))")
).groupBy("community_areas.community").agg(count("*").alias("num_rides"))

result_df = result_df.orderBy(desc("num_rides"))

result_df.show()



AnalysisException: Undefined function: 'ST_Contains'. This function is neither a registered temporary function nor a permanent function registered in the database 'default'.; line 1 pos 0

In [6]:
bucket = "msca-bdp-student-gcs"
spark.conf.set('temporaryGcsBucket', bucket)

# Load data from BigQuery.
df = spark.read.format('bigquery') \
  .option('table', 'msca-bdp-student-ap.chicago_rideshare.community_areas') \
  .load()
df.createOrReplaceTempView('community')

df2 = spark.sql(
    'SELECT * FROM community LIMIT 5')
df2.show()
df2.printSchema()


+-------------+-------+----------+----------+----------+---------+----+--------------------+---------------+---------------+
|    shape_len|comarea|comarea_id|area_numbe|area_num_1|perimeter|area|            geometry|     shape_area|      community|
+-------------+-------+----------+----------+----------+---------+----+--------------------+---------------+---------------+
|31027.0545098|      0|         0|        35|        35|        0|   0|POLYGON((-87.6093...|4.60046211581E7|        DOUGLAS|
|19565.5061533|      0|         0|        36|        36|        0|   0|POLYGON((-87.5921...|1.69139610408E7|        OAKLAND|
|25339.0897503|      0|         0|        37|        37|        0|   0|POLYGON((-87.6288...|1.99167048692E7|    FULLER PARK|
|28196.8371573|      0|         0|        38|        38|        0|   0|POLYGON((-87.6067...|4.84925031554E7|GRAND BOULEVARD|
|23325.1679062|      0|         0|        39|        39|        0|   0|POLYGON((-87.5923...|2.90717419283E7|        KENWOOD|


In [42]:
%%bigquery df

CREATE OR REPLACE TABLE chicago_rideshare.temp2 AS

WITH joined_data AS (
     SELECT 
      COUNT(*) AS num_rides,
      community_areas.community,
      community_areas.area_num_1
     FROM `chicago_rideshare.community_areas` AS community_areas
     INNER JOIN `chicago_rideshare.geo_rides_2018` AS ride_locations 
     ON ST_CONTAINS(community_areas.geometry, ride_locations.pickup_geo)
     GROUP BY community_areas.community, community_areas.area_num_1
   )

SELECT 
    j.num_rides,
    j.community,
    j.area_num_1,
    c.geometry 
FROM joined_data j
   INNER JOIN `chicago_rideshare.community_areas` c
ON j.community = c.community
   ORDER BY j.num_rides DESC

Query is running:   0%|          |

In [43]:
df.head(20)

""


In [44]:
df.write.format("bigquery")\
    .mode('overwrite')\
    .option("table","chicago_rideshare.temp2")\
    .save()

AttributeError: 'DataFrame' object has no attribute 'write'

In [20]:
query = bq_client.query("""
   
   WITH joined_data AS (
     SELECT 
      COUNT(*) AS num_rides,
      community_areas.community,
      community_areas.area_num_1
     FROM `chicago_rideshare.community_areas` AS community_areas
     INNER JOIN `chicago_rideshare.geo_rides_2018` AS ride_locations 
     ON ST_CONTAINS(community_areas.geometry, ride_locations.pickup_geo)
     GROUP BY community_areas.community, community_areas.area_num_1
   )
   
   SELECT 
    j.num_rides,
    j.community,
    j.area_num_1,
    c.geometry 
   FROM joined_data j
   INNER JOIN `chicago_rideshare.community_areas` c
   ON j.community = c.community
   ORDER BY j.num_rides DESC
""")


results = query.result()


In [19]:

from pyspark.sql.functions import col, expr


result_df = spark.createDataFrame(query.to_dataframe()) 

community_areas_df = spark.read.format("bigquery").option("table", "chicago_rideshare.community_areas").load()

community_areas_df = community_areas_df.withColumn(
    "geometry_geojson",
    expr("ST_AsGeoJSON(geometry)")
).select("community", "geometry_geojson")


community_areas_df = community_areas_df.select('community', 'geometry')

final_result_df = result_df.join(
    community_areas_df,
    on="community",
    how="inner"
).orderBy(col("num_rides").desc())


final_result_df.show()

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used


AnalysisException: Undefined function: 'ST_AsGeoJSON'. This function is neither a registered temporary function nor a permanent function registered in the database 'default'.; line 1 pos 0

In [21]:
%%bigquery df2

CREATE OR REPLACE TABLE chicago_rideshare.temp AS
SELECT
  rides.*,
  areas.community, areas.area_num_1, areas.geometry
FROM
  chicago_rideshare.geo_program_area_2020 rides
LEFT JOIN
  chicago_rideshare.community_areas areas
ON
  ST_WITHIN(ST_GEOGPOINT(rides.pickup_lon, rides.pickup_lat), areas.geometry) OR
  ST_WITHIN(ST_GEOGPOINT(rides.dropoff_lon, rides.dropoff_lat), areas.geometry)


Query is running:   0%|          |

In [32]:
sparkDF=spark.createDataFrame(query.to_dataframe()) 


In [24]:
sparkDF.rdd.getNumPartitions()

2

In [34]:
def displaypartitions(df):
    #number of records by partition
    num = df.rdd.getNumPartitions()
    print("Partitions:", num)
    df.withColumn("partitionId", F.spark_partition_id())\
        .groupBy("partitionId")\
        .count()\
        .orderBy(F.asc("count"))\
        .show(num)

sparkDF.rdd.getNumPartitions()
displaypartitions(sparkDF)

Partitions: 2


23/11/17 23:33:27 WARN org.apache.spark.scheduler.TaskSetManager: Stage 19 contains a task of very large size (1030 KiB). The maximum recommended task size is 1000 KiB.


+-----------+-----+
|partitionId|count|
+-----------+-----+
|          0|   38|
|          1|   39|
+-----------+-----+



In [22]:
sparkDF.show()

+---------+---------------+----------+--------------------+
|num_rides|      community|area_num_1|            geometry|
+---------+---------------+----------+--------------------+
|  2829646|NEAR NORTH SIDE|         8|POLYGON((-87.6247...|
|  1478619|           LOOP|        32|POLYGON((-87.6095...|
|  1473545| NEAR WEST SIDE|        28|POLYGON((-87.6383...|
|  1144197|      LAKE VIEW|         6|POLYGON((-87.6439...|
|  1092641|      WEST TOWN|        24|POLYGON((-87.6569...|
|   952820|   LINCOLN PARK|         7|POLYGON((-87.6319...|
|   646320|   LOGAN SQUARE|        22|POLYGON((-87.6828...|
|   366273|         UPTOWN|         3|POLYGON((-87.6410...|
|   322491|NEAR SOUTH SIDE|        33|POLYGON((-87.6091...|
|   298187|          OHARE|        76|MULTIPOLYGON(((-8...|
|   237934|      HYDE PARK|        41|POLYGON((-87.5806...|
|   232537|      EDGEWATER|        77|POLYGON((-87.6557...|
|   209190|   NORTH CENTER|         5|POLYGON((-87.6733...|
|   205838|LOWER WEST SIDE|        31|PO

23/11/17 23:24:29 WARN org.apache.spark.scheduler.TaskSetManager: Stage 9 contains a task of very large size (1030 KiB). The maximum recommended task size is 1000 KiB.


In [35]:
sparkDF=sparkDF.repartition(10)

In [37]:
sparkDF.write.format("bigquery")\
    .mode('overwrite')\
    .option("table","chicago_rideshare.temp2")\
    .option('temporaryGcsBucket', "msca-bdp-student-gcs" )\
    .save()

23/11/17 23:34:10 WARN org.apache.spark.scheduler.TaskSetManager: Stage 29 contains a task of very large size (1030 KiB). The maximum recommended task size is 1000 KiB.


In [36]:
displaypartitions(sparkDF)

23/11/17 23:34:00 WARN org.apache.spark.scheduler.TaskSetManager: Stage 22 contains a task of very large size (1030 KiB). The maximum recommended task size is 1000 KiB.


Partitions: 10


23/11/17 23:34:00 WARN org.apache.spark.scheduler.TaskSetManager: Stage 23 contains a task of very large size (1030 KiB). The maximum recommended task size is 1000 KiB.


+-----------+-----+
|partitionId|count|
+-----------+-----+
|          9|    7|
|          5|    7|
|          0|    7|
|          1|    8|
|          6|    8|
|          3|    8|
|          7|    8|
|          4|    8|
|          8|    8|
|          2|    8|
+-----------+-----+



In [46]:
spark.stop()